In [37]:
import torch
from torchvision import transforms
import tensorflow as tf
import sys
sys.path.append("../")
from models.pytorch_models.model_arch import TransformerNet
import onnx
import onnxruntime

In [30]:
from PIL import Image

def load_image(filename, size=None, scale=None):
    img = Image.open(filename).convert('RGB')
    if size is not None:
        img = img.resize((size, size), Image.ANTIALIAS)
    elif scale is not None:
        img = img.resize((int(img.size[0] / scale), int(img.size[1] / scale)), Image.ANTIALIAS)
    return img


def save_image(filename, data):
    img = data.clone().clamp(0, 255).numpy()
    img = img.transpose(1, 2, 0).astype("uint8")
    img = Image.fromarray(img)
    img.save(filename)


In [ ]:
model_checkpoint = "../models/pytorch_models/rain_princess.pth"
state_dict = torch.load(model_checkpoint)


In [24]:
keys_to_del = []

for i in range(1, 6):
    keys_to_del.extend([
        f"in{i}.running_mean",
        f"in{i}.running_var",
    ])

for i in range(1,3):
    for r in range(1, 6):
        keys_to_del.extend([
            f"res{r}.in{i}.running_mean",
            f"res{r}.in{i}.running_var"
        ])

for k in keys_to_del:
    del state_dict[k]

print(state_dict.keys())

In [25]:
model = TransformerNet()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
model.eval()

In [31]:
# Just some random number
batch_size = 1
height = 224
width = 224

x = torch.randn(batch_size, 3, height, width, requires_grad=True)
output = model(x)

# Convert the PyTorch model to ONNX format
torch.onnx.export(model,                     # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "../models/temp/rain_princess.onnx", # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0: "batch_size", 2: "height", 3: "width"},    # variable lenght axes
                                'output' : {0: "batch_size",  2: "height", 3: "width"}})


In [34]:
def stylize_onnx(model_path, content_image_path, output_image_path):
    """
    Read ONNX model and run it using onnxruntime
    """
    def to_numpy(tensor):
        if tensor.requires_grad:
            return tensor.detach().cpu().numpy()    
        else: 
            return tensor.cpu().numpy()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    assert model_path.endswith(".onnx")
    ort_session = onnxruntime.InferenceSession(model_path)
    
    content_image = load_image(content_image_path, scale=None)
    content_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255))
    ])
    content_image = content_transform(content_image)
    content_image = content_image.unsqueeze(0).to(device)

    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(content_image)}
    ort_outs = ort_session.run(None, ort_inputs)
    img_out_y = torch.from_numpy(ort_outs[0])

    save_image(output_image_path, img_out_y[0])

In [40]:
# Test on any input images you want
stylize_onnx("../models/temp/rain_princess.onnx", "../models/temp/test.jpg", "../models/temp/test_onnx.jpg")

In [42]:
### To TF Freeze Graph (.pb)
!onnx-tf convert -i "../models/temp/rain_princess.onnx" -o  "../models/temp/rain_princess.pb"

2021-03-08 14:52:05,129 - onnx-tf - INFO - Start converting onnx pb to tf pb:
2021-03-08 14:52:05.170554: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-08 14:52:05.173682: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-03-08 14:52:13.573763: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-03-08 14:54:06,881 - onnx-tf - INFO - Converting completes successfully.
INFO:onnx-tf:Converting completes successfully.


In [43]:
### To TF Lite format

converter = tf.lite.TFLiteConverter.from_saved_model('../models/temp/rain_princess.pb')
# tell converter which type of optimization techniques to use
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_model = converter.convert()
# save the converted model 
open('../models/tflite_models/rain_princess.tflite', 'wb').write(tf_lite_model)


1770800